# Инициализация и загрузка данных

In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import LabelEncoder

from tqdm.auto import tqdm, trange

import nltk
import pymorphy2
import re

from lightgbm import LGBMClassifier

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import recall_score, accuracy_score, make_scorer

In [2]:
def seed_everything(seed):
    import os, random
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
SEED = 42
seed_everything(SEED)

In [3]:
df_train = pd.read_csv("data/train.csv", index_col='RecordNo')
df_test = pd.read_csv("data/test.csv", index_col='RecordNo')

### Объединение train и test в одну таблицу

In [4]:
df_train.shape, df_test.shape

((4839, 16), (2075, 11))

In [5]:
df_train['train'] = 1
df_test['train'] = 0

df = pd.concat([df_train, df_test])
del df_train, df_test

In [6]:
df.shape, df.query('train==1').shape, df.query('train==0').shape

((6914, 17), (4839, 17), (2075, 17))

In [7]:
df.head(3)

,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности,train
RecordNo,,,,,,,,,,,,,,,,,
6145,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.7,3922,408,Айгуль Ляпина,5.0,Рекомендую книгу в прочтению/прослушиванию. Ес...,0,3,0.0,0.0,0.0,0.0,0.0,1
7006,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Olga T,5.0,"Удивительно, что сейчас возникает ТАКАЯ литера...",0,1,0.0,0.0,0.0,0.0,1.0,1
1124,Дети мои,Гузель Яхина,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,Кирилл Чириков,5.0,"Душевно, жизненно, чувственно, проникновенно!!...",0,0,0.0,1.0,0.0,0.0,1.0,1


# Обработка признаков

In [8]:
TARGET_COLS = ['Релевантность', 'Таксономия релевантные', 'Таксономия не релевантные', 'Длина отзыва', 'Ценности']

In [9]:
df = df.rename(columns={
    'Отзыв': 'review',
})

**likes_balance**

In [10]:
df['likes_balance'] = df['Лайки на отзыв'] - df['Дислайки на отзыв']

**Оценка читателем** - fill Nans

In [11]:
df['Оценка книги читателем (из 5 баллов)'].fillna(4, inplace=True)

**Автор, Имя читателя, Название книги** - оставлю только те, у кого больше 10 отзывов. И преобразую в числовой код.

In [12]:
print('Осталось уникальных значений:')
for col in ['Автор', 'Имя читателя', 'Название книги']:
    keep_values = set([val for val, count in Counter(df[col]).items() if count > 10])
    df[col] = df[col].apply(lambda x: x if x in keep_values else 'other').astype('category').cat.codes
    print(f'{col} - {df[col].nunique()}')

Осталось уникальных значений:
Автор - 32
Имя читателя - 13
Название книги - 46


### Обработка Отзыва и подсчет длины, числа слов, числа предложений

Число предложений - важный признак, поэтому надо исправить неправильно обрабатываемые случаи, когда пробел стоит до, а не после точки.

In [13]:
def clean_review(s):
    s = s.replace('…', '...')
    s = re.sub(' *([.?!,])+ *', '\\1 ', s)
    return s
    
df['review'] = df['review'].apply(clean_review)

In [14]:
df['review_len'] = df.review.map(len)

df['review_sentences'] = df.review.apply(lambda s: list(nltk.sent_tokenize(s)))
df['review_n_sents'] = df.review_sentences.apply(len)

Выделяю токены слов и нормализую их, приведя в стандартную форму (например "сделаю" -> "сделать" и т.п.).

In [15]:
%%time
df['review_tokens'] = df.review.apply(lambda s: [w.lower() for w in nltk.tokenize.word_tokenize(s) if w.isalpha()])
df['review_n_words'] = df.review_tokens.apply(len)

morph = pymorphy2.MorphAnalyzer()
df['review_norm_tokens'] = df.review_tokens.apply(lambda ws: [morph.parse(w)[0].normal_form for w in ws])
df['review_n_uniq_words'] = df.review_norm_tokens.apply(lambda ws: len(set([w for w in ws if len(w)>2])))

Wall time: 1min 10s


### Счетчик слов (CountVectorizer)

Обрабатываю нормализованные токены.

Полученную матрицу добавляю в pandas таблицу новыми колонками с именами `cvect_N`

In [16]:
df['review_norm'] = df['review_norm_tokens'].apply(lambda x: ' '.join(x))

vect = CountVectorizer(min_df=10)
matrix = vect.fit_transform(df['review_norm']).todense()
vocab_id2word = list(vect.vocabulary_.keys())
col_names = [f'cvect_{i}' for i in range(matrix.shape[1])]
df = pd.concat([df, pd.DataFrame(matrix, index=df.index, columns=col_names)], axis=1)

### Добавление эмбеддинга отзывов

Использую предобученную модель BERT: `rubert-tiny2` с хаба Hugging Face (https://huggingface.co/cointegrated/rubert-tiny2).

In [17]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('тест модели', model, tokenizer).shape)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(312,)


Получение эмбеддинга каждого отзыва

In [18]:
%%time
df['review_embed_rubert_tiny2'] = df.review.apply(lambda s: embed_bert_cls(s, model, tokenizer))

Wall time: 1min 11s


Добавление эмбеддингов в таблицу как новые колонки с именами `emb_N`

In [19]:
embed_column = 'review_embed_rubert_tiny2'  #'review_embed_rubert_tiny2' или 'review_embed_sber'

embeddings = np.array(df[embed_column].values.tolist())
col_names = [f'emb_{i}' for i in range(embeddings.shape[1])]
df = pd.concat([df, pd.DataFrame(embeddings, index=df.index, columns=col_names)], axis=1)

## Выборки

X - вся обучающая выборка, X_train и X_val - это X разбитый на две части. Аналогично с Y, y_train, y_val.

X_test - тестовая выборка для финального предсказания.

In [20]:
object_cols = df.dtypes[df.dtypes == "object"].index.values.tolist()

X = df.query('train==1').drop(object_cols + ['train'] + TARGET_COLS, axis=1)
X_test = df.query('train==0').drop(object_cols + ['train'] + TARGET_COLS, axis=1)
Y = df.query('train==1')[TARGET_COLS]
y_test = df.query('train==0')[TARGET_COLS]

# уберем пробелы из названий колонок, чтобы RF не выдавал предупреждений
X.columns = [c.replace(' ', '_') for c in X.columns]
X_test.columns = [c.replace(' ', '_') for c in X_test.columns]

X_train, X_val, y_train, y_val = train_test_split(X, Y)

# Утилитные функции

In [21]:
def my_metric(true, pred):
    return recall_score(true, pred, average='macro')

my_scorer = make_scorer(recall_score, average='macro')

## Классификатор

Признаки предсказываю независимо, разными моделями.

In [22]:
scores = []
for col in tqdm(TARGET_COLS):
    model = LGBMClassifier(random_state=SEED)
    pred = model.fit(X_train, y_train[col]).predict(X_val)
    score = my_metric(y_val[col], pred)
    scores.append(score)
    print(f'{col} - {score:.4f}')
    
print(f'----\nОбщая метрика: {np.mean(scores):.4f}')

  0%|          | 0/5 [00:00<?, ?it/s]

Релевантность - 0.6537
Таксономия релевантные - 0.9850
Таксономия не релевантные - 0.9286
Длина отзыва - 0.9228
Ценности - 0.9507
----
Общая метрика: 0.8882


### Доработка классификатора

С помощью `Optuna` подобрал гиперпараметры моделей (они отличаются). При подборе для скорости ограничивал датасет признаками с ненулевым feature_importance и нивелировал случайный фактор усреднением по кросс-валидации и повторам с помощью RepeatedKFold.

Как оказалось `Длина отзыва` зависит исключительно от числа предложений в отзыве. Поэтому чтобы не было переобучения на других признаках, оставил для этого таргета только признак `review_n_sents`

Для остальных моделей фильтрации по признакам не делал.

In [25]:
def predict_customized(X, Y, X_test):
    pred = []
    for col in tqdm(TARGET_COLS):
        if col == 'Длина отзыва':
            model = LGBMClassifier(random_state=SEED)
            pred_col = model.fit(X['review_n_sents'].values.reshape(-1,1), Y[col]).predict(X_test['review_n_sents'].values.reshape(-1,1))
        elif col == 'Релевантность':
            params = {'objective': 'binary', 'num_leaves': 177, 'max_depth': 2, 'n_estimators': 260,
                      'learning_rate': 0.05121174906532426, 'min_child_samples': 105, 'unbalanced_sets': True,
                      'reg_alpha': 3.3519079344356215e-08, 'reg_lambda': 4.559393842524269e-05,
                      'colsample_bytree': 0.8034114269054005, 'subsample': 0.4015550377744898, 'subsample_freq': 1,
                     }
            model = LGBMClassifier(random_state=SEED, **params)
            pred_col = model.fit(X, Y[col]).predict(X_test)
        elif col == 'Ценности':
            params = {'objective': 'binary', 'num_leaves': 61, 'max_depth': 5, 'n_estimators': 170, 'learning_rate': 0.09980610611553524, 'min_child_samples': 15, 'unbalanced_sets': False,
                     'reg_alpha': 0.010166786814215246, 'reg_lambda': 0.01470197993560216, 'colsample_bytree': 0.535556342301064, 'subsample': 0.9996493889793364, 'subsample_freq': 5
                     }
            model = LGBMClassifier(random_state=SEED, **params)
            pred_col = model.fit(X, Y[col]).predict(X_test)
        elif col == 'Таксономия не релевантные':
            params = {'objective': 'regression', 'num_leaves': 142, 'max_depth': 5, 'n_estimators': 300,
                      'learning_rate': 0.04688054616706237, 'min_child_samples': 5, 'unbalanced_sets': False}
            model = LGBMClassifier(random_state=SEED, **params)
            pred_col = model.fit(X, Y[col]).predict(X_test)
        elif col == 'Таксономия релевантные':
            params = {'objective': 'binary', 'num_leaves': 7, 'max_depth': 9, 'n_estimators': 270, 'learning_rate': 0.06264444878642536, 'min_child_samples': 15, 'unbalanced_sets': True,
                      'reg_alpha': 0.03294335187156749, 'reg_lambda': 0.0005076535960444901, 'colsample_bytree': 0.7010372380556693, 'subsample': 0.9985674583502681, 'subsample_freq': 2
                     }
            model = LGBMClassifier(random_state=SEED, **params)
            pred_col = model.fit(X, Y[col]).predict(X_test)
        else:
            assert False
        pred.append(pred_col)
    pred = np.array(pred).T
    return pd.DataFrame(pred, index=X_test.index, columns=TARGET_COLS).astype(int)

In [26]:
df_pred = predict_customized(X_train, y_train, X_val)

  0%|          | 0/5 [00:00<?, ?it/s]

In [27]:
for col in TARGET_COLS:
    pred = df_pred[col]
    score = my_metric(y_val[col], pred)
    scores.append(score)
    print(f'{col} - {score:.4f}')
print(f'----\nОбщая метрика: {np.mean(scores):.4f}')

Релевантность - 0.7444
Таксономия релевантные - 0.9913
Таксономия не релевантные - 0.9703
Длина отзыва - 0.9239
Ценности - 0.9569
----
Общая метрика: 0.9028


## Обучение на всем датасете, предсказание для тестовой выборки и экспорт

In [28]:
df_pred = predict_customized(X, Y, X_test)

  0%|          | 0/5 [00:00<?, ?it/s]

In [29]:
filename = 'submission.csv'

In [30]:
df_pred.to_csv(filename)
df_pred

,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности
RecordNo,,,,,
3366,1,0,0,1,1
3952,0,1,0,1,0
6852,1,0,0,0,1
4586,0,0,0,1,1
4677,1,1,0,0,0
...,...,...,...,...,...
4661,0,1,1,0,1
1547,0,0,1,0,0
6071,1,1,0,1,1
